In [1]:
import pandas as pd
import numpy as np

mer = pd.read_json("023_JV_segments_matched_speaker_id_added.jsonl", orient="records", lines=True)
mer = mer[~mer.file.str.contains("_BzZf0fGg0E")]
corp = pd.read_json("020_JV_with_metadata.jsonl", orient="records", lines=True)

def corpus_handler(s:str) -> str:
    """For corpus transcripts."""
    from string import punctuation
    for p in punctuation:
        s = s.replace(p, "")
    return " ".join(
        s.replace("JV:", "").split()
    ).casefold()

def segment_handler(s:str) -> str:
    return s.replace("<anchor_start>", "").replace("<anchor_end>", "")

def find_optimal_subset(segment, full, step=1, limit = None):
    from fuzzywuzzy.fuzz import ratio
    from tqdm.auto import tqdm
    segment = segment.split()
    full = full.split()
    best = 0
    best_start, best_end = 0, -1
    tqdm = lambda x: x
    for start in tqdm(range(0, len(full) if not limit else 2*limit, step)):
        for end in range(len(full)- 2*limit if limit else start, len(full), step):
            try:
                subset = full[start:end]
                current = ratio(
                    segment_handler(" ".join(segment)),
                    corpus_handler(" ".join(subset))
                        )
                # print(start, end, current, subset)
                if current >= best:
                    best_start, best_end = start, end
                    best = current
            except IndexError:
                continue
    if step != 1:
        best_start = max((best_start - step, 0))
        best_end   = min((best_end   + step, len(full)))
    return " ".join(
        full[best_start:best_end]
    )

In [2]:
i = 300
segments_file = mer.file.values[i]
full_transcript = corp[corp.path == segments_file].transcript.values[0]
segments_kaldi_transcript = mer.kaldi_transcript.values[i]

It has been found out that the best course of action is to set up a step-down strategy: instead of matching words to words, it is best to do it first on hecto-words and then only identify word level matches when the best candidate string is far shorter.

Specifically, when doing 100->1 step down, the execution time drops from 5h to 4s, which is a speed-up of more than 36 dB!

Since this was not fast enough, I also implemented a three step version, and the algorithm has been corrected so that after first iteration we only trim the ends of the string.

In [3]:
coarse = find_optimal_subset(segments_kaldi_transcript, full_transcript, step=100)
medium = find_optimal_subset(segments_kaldi_transcript,  coarse, step=10, limit=100)
fine = find_optimal_subset(segments_kaldi_transcript,  medium, step=1, limit=10)

In [4]:
fine

'otklonili te probleme? 364 zdravstvene ustanove postoje u planu mreže u Srbiji. 14 ih ima u Nišu. Ukoliko kupujete 36 olovke, to je po novom zakonu, biće jeftinije nego ako kupujete 1. Isto tako se pokazalo se, pokazalo se i par problema, o kojima ja sada govorim,'

In [5]:
segments_kaldi_transcript

'otklonili te probleme<anchor_end> tristo šezdeset četiri zdravstvene ustanove postoje u planu mreže u srbiji četrnaest ih ima u nišu ukoliko kupujete trideset šest olovke to je po novom zakonu biće jeftinije nego ako kupujete jedan isto tako se pokazalo se pokazalo se i par problema o kojima ja sada govorim'

# running it full scale

In [6]:



def match_kaldi(row):
    try:
        segments_file = row["file"]
        full_transcript = corp[corp.path == segments_file].transcript.values[0]
        segments_transcript = row["kaldi_transcript"]

        coarse = find_optimal_subset(segments_transcript, full_transcript, step=100)
        medium = find_optimal_subset(segments_transcript,  coarse, step=10, limit=100)
        fine = find_optimal_subset(segments_transcript,  medium, step=1, limit=10)
        return fine
    except:
        return ""

from concurrent.futures import ProcessPoolExecutor

with ProcessPoolExecutor(max_workers=32) as executor:
    results = executor.map(match_kaldi, [row for i, row in mer.iterrows()])
mer["raw_transcript__matched_on_kaldi"] = list(results)

In [7]:
mer.to_json("025_segments_matched_with_raw.jsonl", orient="records", lines=True)

In [8]:
def match_asr(row):
    try:
        segments_file = row["file"]
        full_transcript = corp[corp.path == segments_file].transcript.values[0]
        segments_transcript = row["asr_transcription"]

        coarse = find_optimal_subset(segments_transcript, full_transcript, step=100)
        medium = find_optimal_subset(segments_transcript,  coarse, step=10, limit=100)
        fine = find_optimal_subset(segments_transcript,  medium, step=1, limit=10)
        return fine
    except:
        return ""


from concurrent.futures import ProcessPoolExecutor
with ProcessPoolExecutor(max_workers=32) as executor:
    results = executor.map(match_asr, [row for i, row in mer.iterrows()])
mer["raw_transcript__matched_on_asr"] = list(results)

In [9]:
mer.to_json("025_segments_matched_with_raw.jsonl", orient="records", lines=True)

In [10]:
import pandas as pd

mer = pd.read_json("025_segments_matched_with_raw.jsonl", orient="records", lines=True)

In [11]:
mer.columns

Index(['file', 'start', 'end', 'asr_transcription', 'kaldi_transcript',
       'guest_name', 'guest_description', 'host', 'kaldi_words',
       'kaldi_word_starts', 'kaldi_word_ends', 'segment_file',
       'average_distance', 'similarity_ratio', 'speaker_breakdown',
       'Raw_transcript__matched_on_kaldi', 'Raw_transcript__matched_on_asr'],
      dtype='object')

In [13]:
mer[[
'file', 
'segment_file',
'start', 
'end', 
'asr_transcription', 
'kaldi_transcript',
'raw_transcript__matched_on_kaldi', 
'raw_transcript__matched_on_asr',
'guest_name', 
'guest_description', 
'host', 
'kaldi_words',
'kaldi_word_starts',
'kaldi_word_ends', 
'speaker_breakdown',
'average_distance',
'similarity_ratio',
]].to_json("025_segments_matched_with_raw.jsonl", orient="records", lines=True)

In [ ]:
pd.set_option("display.max_colwidth", 100)
mer.sort_values("similarity_ratio")["asr_transcription kaldi_transcript Raw_transcript__matched_on_kaldi Raw_transcript__matched_on_asr".split()]

,asr_transcription,kaldi_transcript,Raw_transcript__matched_on_kaldi,Raw_transcript__matched_on_asr
3418,moje generacija je odrastala slušajući bajki koji je pričao zijaksokolović palčić trno lužica zl...,moje ime je,broj i misle da će,"generacija je odrastala slušajući bajke koje je pričao Zijah Sokolović, Palčić, Trnoružica, Zlat..."
11043,folk svezda svetlana cecera žnatović nastupit će za srpsku novu godinu unišu kakvu poruku ovim v...,folk ne znam da,znao da,Folk zvezda Svetlana Ceca Ražnjatović nastupiće za srpsku novu godinu u Nišu. Kakvu poruku ovim ...
11461,koliko će još dugo deponija bubanj biti u funkciji kako će se otpad u budućnosti odlagati i šta ...,koliko je ljudi upozoreno i koliko je,"ostali mediji, imate sudski proces koji je",Kako će se otpad u budućnosti odlagati i šta će biti sa vanpijačnom prodajom u Nišu? Moje ime je...
8915,naš današnji gos bio je u svim vladama poslije 20. godine ni u jedno i nije ostop po mandat mlađ...,naš gost moje ime je predrag blagojević bila je,osam od deset pitanja je bilo šta je,"današnji gost bio je u svim vladama od 2000. godine i ni u jednoj nije ostao pun mandat, Mlađan ..."
12449,srbiji odavno nije u republitičkom parlamentu predstavljao neko tko koga bar građani ne poznaju ...,srbiji već pita gde će dete da mi ide u,vratiti i da vreme radi samo za Demokratsku,"Srbije odavno nije u republičkom parlamentu predstavljao neko koga, bar građani, ne poznaju samo..."
...,...,...,...,...
11365,mi smo pokušali da preko strukovnih udruženja ukažemo na ovaj problem sva reprezentativna udruže...,mi smo pokušali da preko strukovnih udruženja ukažemo na ovaj problem sva reprezentativna udruže...,Mi smo pokušali da preko strukovnih udruženja ukažemo na ovaj problem. Sva reprezentativna udruž...,Mi smo pokušali da preko strukovnih udruženja ukažemo na ovaj problem. Sva reprezentativna udruž...
349,mnogo sam godina bio novinar aktivno radio i znam kako je bilo u različitim vremenima,mnogo sam godina bio novinar aktivno radio i znam kako je bilo u različitim vremenima,"Mnogo sam godina bio novinar, aktivno radio i znam kako je bilo u različitim vremenima.","Mnogo sam godina bio novinar, aktivno radio i znam kako je bilo u različitim vremenima."
353,redakcija je uglavnom stajala iza svojih novinara i to je nešto što je jako bitno danas mislim d...,redakcija je uglavnom stajala iza svojih novinara i to je nešto što je jako bitno danas mislim d...,"pratim, jednostavno sklapam mozaik iz informacija koje čujem. To nije dobro ali današnje vreme j...","pratim, jednostavno sklapam mozaik iz informacija koje čujem. To nije dobro ali današnje vreme j..."
172,šta je konkretno bio vaš motiv,šta je konkretno bio vaš motiv<anchor_end>,da će možda neko od tih 46 vaših kolegai,da će možda neko od tih 46 vaših kolegai


In [ ]:
mer.Raw_transcript__matched_on_asr.values[8]

'je, da kažem, jedna ustanova. Igrom slucaja je dr Radovanović na čelu te ustanove. Tako je. JV: Vi ste donedavno bili i direktor jednog lokalnog javnog preduzeća. Ali ono što vas pitam jeste, prvenstveno, šta ste vi uradili ili planirate da uradite kao narodni poslanici?'

In [16]:
mer[mer.similarity_ratio > 75].sort_values("similarity_ratio")["asr_transcription kaldi_transcript raw_transcript__matched_on_kaldi".split()].head()

,asr_transcription,kaldi_transcript,raw_transcript__matched_on_kaldi
10147,raspoređeni tako da kad jedan do od 250brojnih...,raspoređeni tako da ako jedan broj prođe u nar...,"raspoređeni, tako da ako jedan broj prođe u na..."
3723,ukoliko se gledamo kružni tok koji nije sinoli...,ukoliko gledamo kružni tok koji nije signalisa...,Ukoliko gledamo kružni tok koji nije signalisa...
3005,nije samo to jedini slučaj podsjetio bih recim...,<anchor_start>nije samo to jedini slučaj podse...,"samo to jedini slučaj, podsetio bi gledaoce na..."
119,krađana drugih zemalja europa naročito što se ...,građana drugih zemalja evrope naročito što se ...,građana drugih zemalja Evrope. Naročito što se...
6215,tu su potencijali u velikoj drvnoj masi vo taj...,tu su potencijali u velikoj drvnoj masi taj ši...,"Tu su potencijali u velikoj drvnoj masi, taj Š..."


In [17]:
ss = mer[mer.similarity_ratio>75]
(ss.end-ss.start).sum() / 3600

68.60344444444445